In [1]:
import uproot 
import awkward as ak
import numpy as np
import vector
import torch
import torch_hep
import torch_geometric
import itertools
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import networkx as nx
from sklearn.model_selection import train_test_split
import sklearn
import mplhep as hep
import boost_histogram as bh
import time
import os
import pandas as pd
from tqdm import tqdm

from torch_geometric.loader import DataLoader


In [2]:
path = 'C:/Users/HP/Documents/data/'
base =  uproot.open(path+"4top_parton_CPstudy.root") 
tree_even = base['CP_even']
tree_odd = base['CP_odd']



In [3]:

dic_even = {}
dic_even['pt'] = np.hstack((tree_even['t_pt'].array(), tree_even['tbar_pt'].array()))
length_even = len(dic_even['pt'])
dic_even['eta'] = np.hstack((tree_even['t_eta'].array(), tree_even['tbar_eta'].array()))
dic_even['phi'] = np.hstack((tree_even['t_phi'].array(), tree_even['tbar_phi'].array()))
dic_even['e'] = np.hstack((tree_even['t_e'].array(), tree_even['tbar_e'].array()))
dic_even['type'] = np.hstack((np.zeros((length_even,2)), np.ones((length_even,2))))


dic_odd = {}
dic_odd['pt'] = np.hstack((tree_odd['t_pt'].array(), tree_odd['tbar_pt'].array()))
length_odd = len(dic_odd['pt'])
dic_odd['eta'] = np.hstack((tree_odd['t_eta'].array(), tree_odd['tbar_eta'].array()))
dic_odd['phi'] = np.hstack((tree_odd['t_phi'].array(), tree_odd['tbar_phi'].array()))
dic_odd['e'] = np.hstack((tree_odd['t_e'].array(), tree_odd['tbar_e'].array()))
dic_odd['type'] = np.hstack((np.zeros((length_odd,2)), np.ones((length_odd,2))))


In [5]:
dataset = []

for i in tqdm(range(length_odd)):
    G = torch_hep.graph.GraphBuilder()
    G.add_asNode(key='x', pt = dic_odd['pt'][i], eta = dic_odd['eta'][i], phi = dic_odd['phi'][i], e = dic_odd['e'][i], type = dic_odd['type'][i],dtype=torch.float32)
    G.add_asEdge(key='edge_attrs', index=list(itertools.permutations(range(len(dic_odd['pt'][i])),2)), dtype=torch.int64)
    G.add_asGlobal(key='y', IsSIG=1, dtype=torch.float32)
    # G.add_asGlobal(key='w', weight=1, dtype=torch.float32)

    dataset.append(G.to_Data())

for i in tqdm(range(length_even)):
    G = torch_hep.graph.GraphBuilder()
    G.add_asNode(key='x', pt = dic_even['pt'][i] ,eta = dic_even['eta'][i], phi = dic_even['phi'][i] , e = dic_even['e'][i], type = dic_even['type'][i],dtype=torch.float32)
    G.add_asEdge(key='edge_attrs', index=list(itertools.permutations(range(len(dic_even['pt'][i])),2)), dtype=torch.int64)
    G.add_asGlobal(key='y', IsSIG=0, dtype=torch.float32)
    # G.add_asGlobal(key='w', weight=0.12, dtype=torch.float32)

    dataset.append(G.to_Data())

100%|██████████| 600000/600000 [02:33<00:00, 3910.58it/s]


In [6]:
seed = 141592
torch.manual_seed(seed)
np.random.seed(seed)
np.random.shuffle(dataset)
#size = 200000
size = len(dataset)
cutoff = int(size*0.8)
#cutoff = 1000

train_dataset = dataset[:cutoff]
test_dataset = dataset[cutoff:size]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')
print(f'percentage of graphs that are signal: {100* length_odd/(length_odd + length_even):.2f}%')


Number of training graphs: 960000
Number of test graphs: 240000
percentage of graphs that are signal: 50.00%


In [7]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=True)
scaler_loader = [i for i in DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=False)][0]
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(scaler_loader.x)
#total_loader = [i for i in DataLoader(dataset, batch_size=len(dataset), shuffle=True)][0]


StandardScaler()

In [14]:
from torch import nn
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv
from torch_geometric.nn import GraphConv
from torch_geometric.nn import GATConv
from torch_geometric.utils import scatter

from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, num_convs):
        super(GCN, self).__init__()
        self.convs = nn.ModuleList()  # Use ModuleList to store layers
        self.convs.append(GCNConv(5, hidden_channels))
        for _ in range(num_convs - 1):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
        self.lin = nn.Linear(hidden_channels, 1)  # Linear layer for final output
        # GraphConv
        # GCNConv


    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings
        for conv in self.convs[:-1]:
            x = conv(x, edge_index)
            x = F.relu(x)
        x = self.convs[-1](x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)
        x = F.dropout(x, p=0.01, training=self.training)
        x = self.lin(x)

        return torch.nn.functional.sigmoid(x)
    def __str__(self):
        return 'GCN'
    
class Graph(torch.nn.Module):
    def __init__(self, hidden_channels, num_convs):
        super(Graph, self).__init__()
        self.convs = nn.ModuleList()  # Use ModuleList to store layers
        self.convs.append(GraphConv(5, hidden_channels))
        for _ in range(num_convs - 1):
            self.convs.append(GraphConv(hidden_channels, hidden_channels))
        self.lin = nn.Linear(hidden_channels, 1)  # Linear layer for final output
        # GraphConv
        # GCNConv


    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings
        for conv in self.convs[:-1]:
            x = conv(x, edge_index)
            x = F.relu(x)
        x = self.convs[-1](x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)
        x = F.dropout(x, p=0.01, training=self.training)
        x = self.lin(x)

        return torch.nn.functional.sigmoid(x)
    def __str__(self):
        return 'GRAPH'
    #GATConv

class GAT(torch.nn.Module):
    def __init__(self, hidden_channels, num_convs):
        super(GAT, self).__init__()
        self.convs = nn.ModuleList()  # Use ModuleList to store layers
        self.convs.append(GATConv(5, hidden_channels))
        for _ in range(num_convs - 1):
            self.convs.append(GATConv(hidden_channels, hidden_channels))
        self.lin = nn.Linear(hidden_channels, 1)  # Linear layer for final output
        # GraphConv
        # GCNConv


    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings
        for conv in self.convs[:-1]:
            x = conv(x, edge_index)
            x = F.relu(x)
        x = self.convs[-1](x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)
        x = F.dropout(x, p=0.01, training=self.training)
        x = self.lin(x)

        return torch.nn.functional.sigmoid(x)
    def __str__(self):
        return 'GAT'
    
class SAGE(torch.nn.Module):
    def __init__(self, hidden_channels, num_convs):
        super(SAGE, self).__init__()
        self.convs = nn.ModuleList()  # Use ModuleList to store layers
        self.convs.append(SAGEConv(5, hidden_channels))
        for _ in range(num_convs - 1):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.lin = nn.Linear(hidden_channels, 1)  # Linear layer for final output
        # GraphConv
        # GCNConv



    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings
        for conv in self.convs[:-1]:
            x = conv(x, edge_index)
            x = F.relu(x)
        x = self.convs[-1](x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)
        x = F.dropout(x, p=0.01, training=self.training)
        x = self.lin(x)

        return torch.nn.functional.sigmoid(x)
    def __str__(self):
        return 'SAGE'

In [15]:
def plot(out_numpy, all_y, zoomed = False):
    hep.style.use(hep.style.ATLAS)
    if zoomed:
        hist1 = bh.Histogram(bh.axis.Regular(50, 0.98, 1)) # Initialises empty histogram with 50 bins spanning [0,500]
        hist2 = bh.Histogram(bh.axis.Regular(50, 0.98, 1))
    else:
        hist1 = bh.Histogram(bh.axis.Regular(5, 0, 1)) # Initialises empty histogram with 50 bins spanning [0,500]
        hist2 = bh.Histogram(bh.axis.Regular(5, 0, 1))
    hist1.fill(out_numpy[all_y==1])    # Fills the histogram with some data
    hist2.fill(out_numpy[all_y ==0])


    hep.histplot([hist1/hist1.size, hist2/hist2.size], label = ['signal', 'background'])
    plt.xlabel('signal confidence')
    plt.ylabel('Normalized Frequency')
    plt.title(f'signal background plot')
    plt.legend()
    #plt.xlim(0,1)
    #plt.savefig(f'using_adam_images\{dim} signal background zoomed.png')
    plt.show()

In [16]:
from torcheval.metrics import BinaryAccuracy
def train_test(model, optimizer, criterion, loader, output = ''):
    if output == 'train':
        model.train()
    else:
        model.eval()
    total_loss = 0
    metric = BinaryAccuracy()
    #out_numpy = np.array([])
    for data in loader:  # Iterate in batches over the training dataset. 
        x_transformed = torch.tensor(scaler.transform(data.x),dtype=torch.float32)
        out = model(x_transformed, data.edge_index, data.batch)  # Perform a single forward pass.
        metric.update(out.flatten(), data.y)
        # loss = torch.sum(criterion(out, data.y.reshape(-1,1))*data.w.reshape(-1,1)) / data.w.sum()# Compute the loss.
        loss = criterion(out, data.y.reshape(-1,1))
        if output == 'train':
            optimizer.zero_grad()  # Clear gradients.   
            loss.backward()  # Derive gradients.
            optimizer.step()  # Update parameters based on gradients.

        total_loss += loss.item() * data.size(0)
    total_loss /= len(loader.dataset)
    accuracy = metric.compute().item()
    if output:
        print(f'{output} accuracy: {100*accuracy:.3f}% , loss: {total_loss:.6f}')
    return model, accuracy, total_loss

In [17]:
def save_comparison(class_type, layers, hidden_dims, train_loss, test_loss, train_acc, test_acc, b_prec, b_recall, s_prec, s_recall, training_time, epoch, learn_rate):
    file_path = 'performance_comparison.csv'
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        print('hi')
    else:
        df = pd.DataFrame()
        print('by')


    # Data to append
    data_to_append = {
        'class type': [class_type.__str__(class_type)],
        'layers': [layers],
        'hidden_dims': [hidden_dims],
        'learn_rate': [learn_rate],
        'training loss' : [train_loss],
        'training accuracy' : [train_acc],
        'test cost' : [test_loss],
        'training accuracy': [train_acc],
        'test accuracy': [test_acc],
        'training time': [training_time],
        'background precision': [b_prec],
        'background recall': [b_recall],
        'signal precision': [s_prec],
        'signal recall': [s_recall],
        'training time per epoch' : [training_time],
        'epochs':[epoch+1]
    }

    df_to_append = pd.DataFrame(data_to_append)
    df = df.dropna(how='all', axis=1)
    df_to_append = df_to_append.dropna(how='all', axis=1)

    # Concatenate the existing DataFrame with the new data
    df = pd.concat([df, df_to_append], ignore_index=True)

    # Write the updated DataFrame back to the CSV file
    df.to_csv(file_path, index=False)

for each epoch:
train accuracy, train loss, train signal rate
test accuracy, test loss, test signal rate


PLAN
------------
- move graphs onto this program
    - roc, accuracy over epochs, cost over epochs

- move the array of cost an accuracy to the initialisation of the model, so it keeps all the training data
- write a general function ot test different numbers of hidden layers, plot results, save results to an excel
- try all above for cp odd and cp even

In [18]:
def plot_signal_background(test_results, train_results, decisions_nn, test_y, class_type, layers, hidden_dims, learn_rate, save=False):
    fpr_nn, tpr_nn, thresholds_nn = sklearn.metrics.roc_curve(test_y, decisions_nn)
    # may need decisions to be 2 dimensional
    class_str = class_type.__str__(class_type)

    plt.plot(fpr_nn, tpr_nn, linestyle="dashed", label="Neural Network")  # plot neural network ROC
    plt.plot([0, 1], [0, 1], linestyle="dotted", color="grey", label="Luck")  # plot diagonal line to indicate luck
    plt.xlabel("False Positive Rate")  # x-axis label
    plt.ylabel("True Positive Rate")  # y-axis label
    plt.ylim(0,1)
    plt.title(f'{class_str}, layers {layers}, dims {hidden_dims} {learn_rate} ROC plot')
    plt.grid()  # add a grid to the plot
    plt.legend()  # add a legend
    if save:
        plt.savefig(f'plots/{class_str}, layers {layers}, dims {hidden_dims} {learn_rate} ROC plot.png')
    plt.show()


    plt.plot(np.arange(len(train_results)), train_results[:,1],linestyle='-', marker='o',  label = 'train loss')
    plt.plot(np.arange(len(test_results)), test_results[:,1], linestyle='-', marker='o', label = 'test loss')
    plt.xlabel("Epochs")  # x-axis label
    plt.ylabel("Loss")  # y-axis label
    plt.title(f'Loss at each Epoch {class_str}, layers {layers}, dims {hidden_dims} {learn_rate}')
    plt.legend()
    if save:
        plt.savefig(f'plots/{class_str}, layers {layers}, dims {hidden_dims} {learn_rate} loss at each epoch.png')
    plt.show()

    plt.plot(np.arange(len(train_results)), train_results[:,0],linestyle='-', marker='o',  label = 'train accuracy')
    plt.plot(np.arange(len(test_results)), test_results[:,0], linestyle='-', marker='o', label = 'test accuracy')
    plt.xlabel("Epochs")  # x-axis label
    plt.ylabel("Percentage Accuracy")  # y-axis label
    plt.title(f'Accuracy {class_str}, layers {layers}, dims {hidden_dims} {learn_rate}')
    plt.legend()
    if save:
        plt.savefig(f'plots/{class_str}, layers {layers}, dims {hidden_dims} {learn_rate} accuracy at each epoch.png')
    plt.show()


    hep.style.use(hep.style.ATLAS)

    hist1 = bh.Histogram(bh.axis.Regular(70, 0.3, 0.7)) # Initialises empty histogram with 50 bins spanning [0,500]
    hist2 = bh.Histogram(bh.axis.Regular(70, 0.3, 0.7))
    hist1.fill(decisions_nn[test_y==1])    # Fills the histogram with some data
    hist2.fill(decisions_nn[test_y ==0])

    hep.histplot([hist1/hist1.size, hist2/hist2.size], label = ['signal', 'background'])
    plt.xlabel('signal confidence')
    plt.ylabel('Normalized Frequency')
    plt.title(f'signal background plot {class_str}, layers {layers}, dims {hidden_dims} {learn_rate}')
    plt.legend()
    #plt.xlim(0,1)
    if save:
        plt.savefig(f'plots/{class_str}, layers {layers}, dims {hidden_dims} {learn_rate} signal background.png')
    plt.show()



In [20]:
def each_run(class_type, criterion, layers, hidden_dims, learn_rate, num_epochs):
    model = class_type(hidden_channels=hidden_dims, num_convs=layers)
    optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)
    test_results =  np.empty((0,2))
    train_results =  np.empty((0,2))
    t1 = time.perf_counter()
    for epoch in range(num_epochs[1]):
        print(f'Epoch: {epoch:03d}')
        model, acc, loss = train_test(model, optimizer, criterion, train_loader, 'train')
        train_results = np.vstack((train_results, [acc,loss]))
        model, acc, loss = train_test(model, optimizer, criterion, test_loader, 'test')
        test_results = np.vstack((test_results, [acc,loss]))
        print()
        if epoch>num_epochs[0]:
            if (np.max(train_results[:,1][-5:-1]) - train_results[:,1][-1]) <0:
                break
    time_per_epoch = (time.perf_counter()-t1)/(epoch+1)
    test_out = np.array([]) # first element is prediction, last element is what it should be
    test_y = np.array([])
    for data in test_loader:
        x_transformed = torch.tensor(scaler.transform(data.x),dtype=torch.float32)
        out = model(x_transformed, data.edge_index, data.batch).cpu().detach().numpy()[:,0]
        test_out = np.hstack((test_out, out))
        test_y = np.hstack((test_y, np.array(data.y)))
    y_pred_test = (test_out>0.5)*1
    
    print(sklearn.metrics.classification_report(test_y, y_pred_test, target_names=["background", "signal"]))
    report = sklearn.metrics.classification_report(test_y, y_pred_test, target_names=["background", "signal"]).split()
    b_prec, b_recall = report[5:7]
    s_prec, s_recall = report[10:12]
    save_comparison(class_type, layers, hidden_dims, train_results[-1][1], test_results[-1][1], 
                    train_results[-1][0], test_results[-1][0], b_prec, b_recall, s_prec, s_recall, time_per_epoch, epoch, learn_rate)
    # cannot just pass in the test out since the loader is shuffled each epoch
    plot_signal_background(test_results, train_results, test_out, test_y, class_type, layers, hidden_dims, learn_rate, save = True)
    torch.save(model.state_dict(), f'models\model {class_type.__str__(class_type)} {layers} {hidden_dims} {epoch+1} {0.001}.pth')


TODO
- why is the test loss lower than the training (especially since the signal confidence plot is so much worse)
    - could be due to how loss is defined - cannot be dependant on the size
- signal rate seems to disagree wiht the visual confidence


BEST NETWORKS
- GCN   3 64 (>40)
- GCN   3 128 (>40)
- GCN   4 16 (>40)
- GRAPH 4 8 (>40)
- GAT   1 32 (>40) (BAD SIGANL RECALL)
- GAT   1 64 (>40)
- GAT   2 32 (>40)
- GAT   3 128 (34)
- GAT   4 128 (31) (BEST) 89, 90, 14, 13
- GAT   5 128 (21)
- SAGE  5 128 (21) (BEST) 89, 90, 14, 13

In [21]:
# class_list = [GCN, Graph, GAT, SAGE]
# layers_list = [1,2,3,4,5]
# hidden_dims_list = [8,16,32,64,128,256]
# class_list = [SAGE]
# layers_list = [4]
# hidden_dims_list = [64]

schedule = [[SAGE, 4, 64, 0.0001], [SAGE, 4, 64, 0.001]]

#optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.8)
criterion = torch.nn.BCELoss(reduction = 'mean')


def do_it_all(class_list, layers_list, hidden_dims_list, num_epochs, learn_rate):
    for class_type in class_list:
        for layers in layers_list:
            for hidden_dims in hidden_dims_list:
                print(f'{class_type.__str__(class_type)} {layers} {hidden_dims} {learn_rate} ')
                each_run(class_type, criterion, layers, hidden_dims, learn_rate, num_epochs)

def do_it_all_specific(schedule, num_epochs):
    for process in schedule:
        print(f'{process[0].__str__(process[0])} {process[1]} {process[2]} {process[3]} ')
        each_run(process[0], criterion, process[1], process[2], process[3], num_epochs)
                
                

In [22]:
do_it_all_specific(schedule, (25,60))

SAGE 4 64 0.0001 
train accuracy: 52.818% , loss: 2.763606
test accuracy: 52.967% , loss: 2.762600

train accuracy: 52.984% , loss: 2.762345
test accuracy: 52.840% , loss: 2.763462

train accuracy: 53.145% , loss: 2.761030
test accuracy: 53.053% , loss: 2.761122

train accuracy: 53.277% , loss: 2.759714
test accuracy: 53.477% , loss: 2.758525

train accuracy: 53.422% , loss: 2.758678
test accuracy: 53.474% , loss: 2.757629

train accuracy: 53.502% , loss: 2.758109
test accuracy: 53.605% , loss: 2.757202

train accuracy: 53.587% , loss: 2.757562
test accuracy: 53.580% , loss: 2.756406

train accuracy: 53.599% , loss: 2.756961
test accuracy: 53.612% , loss: 2.756264

train accuracy: 53.695% , loss: 2.756386
test accuracy: 53.579% , loss: 2.757168

train accuracy: 53.714% , loss: 2.755948
test accuracy: 53.609% , loss: 2.756563

train accuracy: 53.789% , loss: 2.755411
test accuracy: 53.773% , loss: 2.754980

train accuracy: 53.843% , loss: 2.754902
test accuracy: 53.809% , loss: 2.754792

In [16]:
class_type = SAGE
layers = 4
hidden_dims = 64


model = class_type(hidden_channels=hidden_dims, num_convs =layers)
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
#optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.8)
criterion = torch.nn.BCELoss(reduction = 'mean')


test_results =  np.empty((0,2))
train_results =  np.empty((0,2))

In [ ]:

for epoch in tqdm(range(30)):
    print(f'Epoch: {epoch:03d}')
    train_results = np.vstack((train_results, train_test(train_loader,scaler, 'train')))
    test_results = np.vstack((test_results, train_test(test_loader,scaler, 'test')))
    print()

In [ ]:
test_out = np.array([]) # first element is prediction, last element is what it should be
test_y = np.array([])
x_transformed = torch.tensor(scaler.transform(data.x),dtype=torch.float32)
out = model(x_transformed, data.edge_index, data.batch).cpu().detach().numpy()[:,0]
test_out = np.hstack((test_out, out))
test_y = np.hstack((test_y, np.array(data.y)))
y_pred_test = (test_out>0.5)*1

print(sklearn.metrics.classification_report(test_y, y_pred_test, target_names=["background", "signal"]))


In [ ]:
plot_signal_background(test_results, train_results, test_out, test_y, class_type, layers, hidden_dims)